In [1]:
# main page
# extract parameters from the first block 
# 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pandas as pd 
from itertools import chain 
import numpy as np 

In [2]:
def clean (df): 
    #To int: 
    def to_int (item):
        a =[]
        for n in item:
            if n.isnumeric():
                a.append(n)
        return int(''.join(a)) 
    
    df['slutpris'] = df['slutpris'].apply(to_int)
    df['city-kommun'] = df['område'].apply(lambda i :  i.split()[-1] )
    df['type of boende'] = df['område'].apply(lambda i :  i.split()[0] )
    df['area'] = df['område'].apply(lambda row : row.split()[1].strip(','))
    df['size'] = df['size'].apply(lambda x: float(x.split()[0].replace(',', '.')) if len(x) >30 else np.nan)
    df['price_change'] = df['price_change'].apply(lambda x: x.split()[0] if type(x) != float and len(x)!=1 else np.nan)
    df['price_change'] = df['price_change'].apply(lambda x: int(x.strip('-')) if type(x) != float else np.nan)
    df['pris_per_m2'] = round(df['slutpris'] / df['size'],0)
    cols_to_drop = ['område']
    df.drop(cols_to_drop,axis=1,inplace=True)
    df = df[['gata','city-kommun','type of boende','area', 'slutpris', 'pris_per_m2', 'size', 'price_change']]
    return df 

def hemnet_generator(sold_age = '6m',zone = None,num_pages = 3, keyword = None): 
    adress_ = [] 
    adress2_ = []
    size = [] 
    tomt_size = [] 
    slutpris_ = [] 
    pris_per_m2_ = [] 
    price_change = [] 
    
    for n in range(1,num_pages + 1 ):
        järfälla_url = f"https://www.hemnet.se/salda/bostader?location_ids%5B%5D={zone}&page={n}&sold_age={sold_age}"
        for page in järfälla_url:
            b = ''.join(järfälla_url)
        page = requests.get(b)
        soup = BeautifulSoup(page.text, "html.parser")
        
        location = soup.findAll('div', class_='sold-property-listing__location')
        price = soup.findAll('div', class_='sold-property-listing__price')
        main_size = soup.findAll('div', class_='sold-property-listing__size')
        fee1 = soup.find_all('div', class_= "sold-property-listing__fee")
        
        adr = [i.find('span', class_= 'item-result-meta-attribute-is-bold item-link').text for i in location]
        adr2 =  [i.find('div').text for i in location]
        sp =[i.find('span', class_= 'sold-property-listing__subheading sold-property-listing--left').text for i in price]
        si = [i.find('div').text for i in main_size]
        p_m2 =[i.find('div', class_= 'sold-property-listing__price-per-m2 sold-property-listing--left') for i in price ] 
    

        for i in soup.findAll('div', class_='sold-property-listing'): 
            pc = i.find('div', class_='sold-property-listing__price-change')
            if pc == None:
                price_change.append(np.nan)
            else:
                price_change.append(pc.text)
    
        for row in soup.findAll('div', class_='sold-property-listing__size'): 
            a = row.find('div', class_='sold-property-listing__subheading')
            size.append(a.text)
        
        ##### To lists:
        adress_.append(adr)
        adress2_.append(adr2)
        slutpris_.append(sp)
        pris_per_m2_.append(p_m2)
        
    ##### Flatten arrays 
    adress = list(chain.from_iterable(adress_))
    pris_per_m2 = list(chain.from_iterable(pris_per_m2_))
    slutpris = list(chain.from_iterable(slutpris_))
    adress2 = list(chain.from_iterable(adress2_))
        
    d = {
        'gata': adress, 
        'område':adress2, 
        'slutpris':slutpris,
        'pris_per_m2':pris_per_m2,
        'size':size,
        'price_change':price_change
        }
    df = pd.DataFrame(d)
    df = clean (df)

    #Testing
    if keyword != None :
        kw_list = []
        for row in df.gata: 
            if keyword in row:
                kw_list.append(row)
        return df[df["gata"].isin(kw_list)] 
               
    else:
        return df   

### To dataframe

In [3]:
########
#Loc
########

järfälla = 17951
sollentuna = 18027
solna = 18028
sundbyberg = 18042
hammarby_sjöstad =925972
johhaneshov =473376
kungsholmen = 925968

In [198]:
jak_deps_6m = hemnet_generator(zone = järfälla  ,num_pages=50, keyword='Vasavägen')
jak_deps_3m = hemnet_generator(sold_age='3m' ,zone = järfälla  ,num_pages=50, keyword='Vasavägen')

### EDA

In [93]:
########
#Brainstorm
########

#correlation with price 
#create a filter based on my needs to see where I could buy - create a function that calculates this based on the price increases

In [4]:
# Zones I am interested in:
hammarby_s = hemnet_generator(zone = hammarby_sjöstad  ,num_pages=50)
sollentuna = hemnet_generator(zone = sollentuna  ,num_pages=50)
solna = hemnet_generator(zone = solna ,num_pages=50)
sundbyberg = hemnet_generator(zone = sundbyberg ,num_pages=50)
johhaneshov = hemnet_generator(zone = johhaneshov,num_pages=50)
kungsholmen = hemnet_generator(zone = kungsholmen, num_pages=50)
järfälla = hemnet_generator(zone = järfälla, num_pages=50)

In [16]:
järfälla.head()

,gata,city-kommun,type of boende,area,slutpris,pris_per_m2,size,price_change
0,Grönstensvägen 1,Järfälla,Villa,Skälby,6225000,65526.0,95.0,4.0
1,Tröskvägen 101,Järfälla,Par/Rad/Kedjehus,Viksjö,4100000,30827.0,133.0,8.0
2,Legendvägen 13,Järfälla,Bostadsrättslägenhet,Kallhäll,1600000,39506.0,40.5,7.0
3,Vendelgränd 11,Järfälla,Bostadsrätt,Järfälla,5020000,36642.0,137.0,5.0
4,Ålstavägen 19E,Järfälla,Bostadsrättslägenhet,Barkarbystaden,2250000,32609.0,69.0,13.0


In [26]:
#############
#avg price per m2 per kommun
#############

all_zones = [hammarby_s,sollentuna,solna ,sundbyberg,johhaneshov,kungsholmen,järfälla,]
info = {} 
for idx,zone in enumerate(all_zones):
    p_m2 = round(zone['pris_per_m2'].mean(),2)
    p_c = round(zone['price_change'].mean(),2)
    info.setdefault(idx, []).append([p_m2,p_c]) 
info

{0: [[70825.38, 6.52]],
 1: [[43056.83, 6.95]],
 2: [[58225.56, 12.62]],
 3: [[57490.76, 10.02]],
 4: [[69085.97, 15.35]],
 5: [[90569.64, 10.07]],
 6: [[35905.07, 7.97]]}

In [ ]:
#CONTINUE.....

In [294]:
x = hammarby_s[(hammarby_s['size']>=50) & (hammarby_s['size']<=60) ]
x.describe()

,slutpris,pris_per_m2,size,price_change
count,3.800000e+01,38.000000,38.000000,29.000000
mean,3.733684e+06,68223.815789,54.802632,7.034483
std,3.567471e+05,6467.708011,2.851125,5.685085
min,3.250000e+06,55378.000000,50.000000,1.000000
25%,3.431250e+06,63325.500000,53.625000,3.000000
50%,3.600000e+06,68000.000000,55.250000,5.000000
75%,3.900000e+06,71975.000000,56.875000,10.000000
max,4.650000e+06,86111.000000,59.500000,26.000000


In [304]:
def dep_filter(zone,size=55,max_price=4000000, min_price = 3000000):
    df = hemnet_generator(zone = zone,num_pages=50)
    df = df[(df['size']>=size) & (df['slutpris']<=max_price) & (df['slutpris']>=min_price) ]
    return df


In [305]:
sollentuna = 18027
solna = 18028
sundbyberg = 18042
hammarby_sjöstad =925972
johhaneshov =473376
kungsholmen = 925968

sund = dep_filter(sundbyberg)
sollentuna = dep_filter(sollentuna)
solna = dep_filter(solna)
hammar= dep_filter(hammarby_sjöstad)
johhaneshov = dep_filter(johhaneshov)
kungsholmen  = dep_filter(kungsholmen )


In [307]:
sund.shape

(117, 8)

In [310]:
sollentuna.shape

(89, 8)

In [315]:
kungsholmen 

,gata,city-kommun,type of boende,area,slutpris,pris_per_m2,size,price_change
647,"Lindhagensgatan 69, vån 1",Stockholm,Bostadsrättslägenhet,Kungsholmen,3800000,67857.0,56.0,15.0
668,Gjutargatan 6,Stockholm,Bostadsrättslägenhet,Kungsholmen,3975000,72273.0,55.0,NaN


# Appendix

In [ ]:
#Check floats:
for n in df['size']:
    if type(n) != float: 
        
        print(n)

In [64]:
# Conversion func 1
#Temp solution: func 2
# l = [i.split()[:2] for i in df['pris_per_m2'] ]
# pris_per_m2 = [ int(''.join(i))for i in l]
# len(pris_per_m2) 

In [ ]:
def group (df,g1,metric, mean=True, head= 15):
    if mean:
        return pd.DataFrame(df.groupby(g1)[metric].mean()).sort_values(by=metric,ascending=False).head(head)
    else:
        return pd.DataFrame(df.groupby(g1)[metric].sum()).sort_values(by=metric,ascending=False).head(head)